# Chuẩn bị



1. Tải dataset từ https://www.kaggle.com/code/gabrielhaselhurst/chess-dataset/input

2. Di chuyển file này vào thư mục chess-bot

## Import thư viện cần thiết

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import chess
import chess.svg
import time
from typing import List
import pandas as pd
from IPython.display import display, SVG, HTML
import math
from evaluation.static_evaluation import evaluate_board
from bot import ChessBot
import sys
import os


[8, 9, 16, 17]


## Lấy tập dữ liệu

In [2]:
df = pd.read_csv('random_evals.csv')
df.head()
test_fens = df.sample(n=10, random_state=10)['FEN'].to_list()#should be 15-25 but to slow
last_test_fens = df.sample(n=20, random_state=10)['FEN'].to_list()#for the 3 best weights each generation 

## Visualization

In [3]:
def showBoard(fens, boards_per_row=5):
    total_boards = len(fens)
    rows = math.ceil(total_boards / boards_per_row)
    
    for row in range(rows):
        boards_in_row = []
        for col in range(boards_per_row):
            idx = row * boards_per_row + col
            if idx < total_boards:
                board = chess.Board(fens[idx])
                svg = chess.svg.board(board, size=200)  # Giảm size nếu cần
                boards_in_row.append(SVG(svg)._repr_svg_())  # Lấy chuỗi SVG
        
        # Tạo HTML với flexbox để xếp hàng ngang
        html = f"""
        <div style="display: flex; flex-wrap: nowrap; gap: 10px; margin: 10px 0;">
            {''.join(boards_in_row)}
        </div>
        """
        display(HTML(html))

In [4]:
showBoard(test_fens)

# 1. Initialization

In [5]:
def initialize_population(size: int):
    """
    Initialize a population with chess-specific value ranges.
    
    Args:
        size (int): Number of individuals in the population.

    Returns:
        np.ndarray: Population matrix (size, num_params) with meaningful bounds.
    """
    # Define bounds for each gene type
    bounds = np.array([
    [150, 550],           # square_controlled_by_opponent_pawn_penalty (350 ± 200)
    [0, 200],            # captured_piece_value_multiplier (100 ± 100)

    [90_00, 110_00],   # hash_move_score (100k ± 10%)
    [6_00, 10_00],     # winning_capture_bias (8K ± 2K)
    [5_00, 7_00],      # promote_bias (6K ± 1K)
    [3_00, 5_00],      # killer_bias (4K ± 1K)
    [1_50, 2_00]       # losing_capture_bias (2K ± 500)
    ], dtype=int)

    num_params = len(bounds)

    # Generate population matrix
    population = np.zeros((size, num_params))

    for i in range(num_params):
        low, high = int(bounds[i][0]), int(bounds[i][1])
        population[:, i] = np.random.randint(low, high + 1, size=size)
    return population

# Generate population with 200 individuals
population = initialize_population(20)

# Print first 3 individuals for verification
print(population[:3])


[[  528.   194. 10659.   797.   627.   301.   196.]
 [  470.   115. 10275.   779.   661.   437.   197.]
 [  447.   114. 10094.   811.   558.   437.   174.]]


# Fitness function

In [6]:
def fitness_function(weights: List[float]=None,nnue_order: bool = False,depth: int = 6,use_time=False) -> float:
    """
    caculate fitness by time searching

    Args:
        weights (List[float]): weight in move ordering 
        nnue_order(bool): does this weight is for nnue order 
        depth (int): the depth to caculate fitness 5 for normal caculate, over 7 for last result 

    Returns:
        np.array: Selected individual."""
    try:
        #Chuyển hướng stdout(ẩn print trong find best move)
        stdout_goc = sys.stdout
        sys.stdout = open(os.devnull, 'w')      

        #bắt đầu tính toán fitness 
        total_fitness = 0
        if(use_time):
            start = time.perf_counter()

            for fen in test_fens:
                if nnue_order:
                    chess_bot = ChessBot(use_nnue=True)
                else:
                    chess_bot = ChessBot()   

                if weights is not None:
                    chess_bot.searcher.move_orderer.update_weights(weights) #chuyển các trọng số đang xét vào 
                chess_bot.set_position(fen)
                chess_bot.get_best_move(depth=depth) # nói thật chỗ này t chưa chắc, tốt nhất là chạy ở 5,6,7 + chưa chắc dùng đúng hàm kk
            
            #khôi phục std out
            sys.stdout = stdout_goc
            end = time.perf_counter()
            total_fitness= (end - start) * 1000 #fitness = tổng thời gian search;
            
            return total_fitness / len(test_fens) # thời gian càng ít càng tốt
        else:
            for fen in test_fens:
                if nnue_order:
                    chess_bot = ChessBot(use_nnue=True)
                else:
                    chess_bot = ChessBot()   

                if weights is not None:
                    chess_bot.searcher.move_orderer.update_weights(weights) #chuyển các trọng số đang xét vào 
                chess_bot.set_position(fen)
                chess_bot.get_best_move(depth=depth) # nói thật chỗ này t chưa chắc, tốt nhất là chạy ở 5,6,7 + chưa chắc dùng đúng hàm kk
                total_fitness += chess_bot.searcher.get_node_searchered()

            #khôi phục std out
            sys.stdout = stdout_goc
            print(f"total fitness: {total_fitness}")
            return total_fitness / len(test_fens) # thời gian càng ít càng tốt
    except Exception as e:
        print("bug exist, return inf")
        return np.inf

# Selection

In [7]:

def tournament_selection(population, fitness_scores, tournament_size=4):
    """
    Selects a parent using tournament selection.

    Args:
        population (np.array): Population of individuals.
        fitness_scores (np.array): Fitness scores of individuals.
        tournament_size (int): Number of individuals in the tournament.

    Returns:
        np.array: Selected individual.
    """
    selected_indices = np.random.choice(len(population), tournament_size, replace=False)
    best_index = selected_indices[np.argmin(fitness_scores[selected_indices])]
    return population[best_index]

# Crossover

In [8]:
def crossover_mixed(parent1, parent2, alpha=0.4, crossover_rate=0.5):
    """
    Perform crossover with an adaptive probability.

    Args:
        parent1 (np.array): First parent.
        parent2 (np.array): Second parent.
        alpha (float): Alpha parameter for BLX-α crossover (for floats).
        crossover_rate (float): Probability of crossover.

    Returns:
        np.array: Child with mixed-type values.
    """
    p1, p2 = np.array(parent1), np.array(parent2)
    child = np.zeros_like(p1)

    for i in range(len(p1)):
        if np.random.rand() < crossover_rate:  # Perform crossover with given probability
            min_val, max_val = min(p1[i], p2[i]), max(p1[i], p2[i])
            range_val = max_val - min_val
            child[i] = min_val - alpha * range_val + np.random.rand() * (1 + 2 * alpha) * range_val
        else:
            child[i] = np.random.choice([p1[i], p2[i]])  # Inherit directly from a parent

        child[i] = int(child[i])
        
    return child


In [9]:
def binary_mutate_gen(gen1, gen2):
    #use a random bit mark to 
    min_int=-1<<30
    max_int=(1<<30)+1
    gen1 = int(gen1)
    gen2 = int(gen2)
    mark = np.random.randint(min_int,max_int)
    return (mark & mark) + ((~mark) & gen2)

In [10]:
def binary_mutated_crossover(parent1, parent2, crossover_rate=0.15):
    """
    Perform crossover with binary mutate.
    Args:
        parent1 (np.array): First parent.
        parent2 (np.array): Second parent.
        crossover_rate (float): Probability of mutate decress by generate to 0.

    Returns:
        np.array: Child with mixed-type values.
    """
    p1, p2 = np.array(parent1), np.array(parent2)
    child = np.zeros_like(p1)

    for i in range(len(p1)):
        if np.random.rand() < crossover_rate:  # Perform crossover with given probability
            child[i] =  binary_mutate_gen(p1[i], p2[i])
        else:
            child[i] = np.random.choice([p1[i], p2[i]])  # Inherit directly from a parent
        child[i] = int(child[i])

    return child

# Mutation

In [11]:
 


def mutate_mixed(individual, mutation_rate=0.2, mutation_strength=0.05):
    """
    Apply Mutation integer genes.

    Returns:
        np.array: Mutated individual.
    """
    mutated = np.copy(individual)

    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:  # Mutation occurs with probability
            step = np.random.randint(-1, 2)  # Change by -1, 0, or +1
            mutated[i] += step*mutated[i] * mutation_strength;
            mutated[i] = int(mutated[i])  # chuyển về số nguyên
    return mutated




In [12]:
def adaptive_binary_mutation_rate(generation, max_mutate_generations, max_rate=0.5):
    """
    Adaptive mutation rate: Decreases as generations progress.
    """
    if(generation>max_mutate_generations): return 0
    return max_rate - max_rate * (generation / max_mutate_generations)

In [13]:
def adaptive_mutation_rate(generation, max_generations, min_rate=0.05, max_rate=0.3):
    """
    Adaptive mutation rate: Decreases as generations progress.
    """
    return max_rate - (max_rate - min_rate) * (generation / max_generations)

In [14]:

def adaptive_crossover_rate(generation, max_generations, max_binary_crossover_generation, min_rate=0.2, max_rate=0.7):
    """
    Adaptive crossover rate: Increases as generations progress.
    """
    if(generation < max_binary_crossover_generation): 
        return adaptive_binary_mutation_rate(generation, max_binary_crossover_generation)
    
    return min_rate + (max_rate - min_rate) * (generation / max_generations)


In [15]:
def adaptive_mutation_strength(generation, max_generations, min_strength=0.2, max_strength=1):
    """
    Adaptive mutation strength: Decreases as generations progress.
    first it will be high to make strong dif in weights
    """
    return max_strength - (max_strength - min_strength) * (generation / max_generations)

# GA

In [ ]:
def genetic_algorithm(
    fitness_function,
    population_size=40,
    generations=100,
    elite_size=8,
    tournament_size=5,
    convergence_threshold=20,
    max_binary_crossover_generation=5
):
    """
    Runs an optimized Genetic Algorithm with adaptive mutation and crossover rates.

    Args:
        fitness_function (function): Function to evaluate individuals.
        population_size (int): Number of individuals.
        generations (int): Max number of generations.
        elite_size (int): Number of top individuals retained.
        tournament_size (int): Tournament selection group size.
        convergence_threshold (int): Stop early if no improvement.

    Returns:
        np.ndarray: Best found individual.
        list: Fitness score history.
    """
    population = initialize_population(population_size)
    best_fitness_history = []
    best_fitness = np.inf
    no_improvement_count = 0

    chess_bot = ChessBot()#for save weight
    defaulse_fitness=fitness_function()
    for generation in range(generations):
        # Adaptive rates
        mutation_rate = adaptive_mutation_rate(generation, generations)
        crossover_rate = adaptive_crossover_rate(generation, generations, max_binary_crossover_generation)
        mutation_strength=adaptive_mutation_strength(generation,generations)
        # Evaluate fitness for all individuals
        fitness_scores = np.array([fitness_function(ind) for ind in population])

        # Sort and keep top elite individuals
        sorted_indices = np.argsort(fitness_scores)
        elite = population[sorted_indices[:elite_size]]
        
        # Track best fitness
        current_best_fitness = fitness_scores[sorted_indices[0]]
        best_fitness_history.append(current_best_fitness)
        for i in range(0,5):
            path = f"search/save_moveordering_weights{generation}.{i}.json"
            chess_bot.searcher.move_orderer.update_weights(population[i])
            chess_bot.searcher.move_orderer.save_weights(path)
        # Check if fitness improved
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            no_improvement_count = 0

        else:
            no_improvement_count += 1

        # Stop early if fitness has not improved
        if no_improvement_count >= convergence_threshold:
            print(f"✅ Converged after {generation+1} generations! Stopping early.")
            break

        # Generate new population
        new_population = []
        for _ in range(population_size - elite_size):
            p1 = tournament_selection(population, fitness_scores, tournament_size)
            p2 = tournament_selection(population, fitness_scores, tournament_size)

            if(generation < max_binary_crossover_generation):
                child = binary_mutated_crossover(p1, p2, crossover_rate=crossover_rate)
            else:
                child = crossover_mixed(p1, p2, crossover_rate=crossover_rate)
                
            child = mutate_mixed(child, mutation_rate, mutation_strength=mutation_strength)
            new_population.append(child)

        # Create next generation
        population = np.vstack((elite, new_population))

        print(f"Generation {generation+1}: Best Fitness = {current_best_fitness:.3f}, Mutation Rate = {mutation_rate:.3f}, Crossover Rate = {crossover_rate:.3f}")
        print(f"better compare with defaulse {defaulse_fitness/current_best_fitness} time.")

    return population[0], best_fitness_history


# Tuning

In [ ]:
best_weights, best_fitness_history = genetic_algorithm(fitness_function)

# Plot Convergence Graph
plt.plot(best_fitness_history, label="Best Fitness")
plt.xlabel("Generation")
plt.ylabel("Fitness Score")
plt.title("Genetic Algorithm Convergence")
plt.legend()
plt.show()

print("Optimized Weights:", best_weights)

chromosomes = [            ]
print("Suggested weights fitness:", fitness_function(chromosomes))

Initializing searcher
total fitness: 309047
total fitness: 315597
total fitness: 318584
total fitness: 316851
total fitness: 310548
total fitness: 310342
total fitness: 307002
total fitness: 307744
total fitness: 312724
total fitness: 318061
total fitness: 310432
total fitness: 310453
total fitness: 309567
total fitness: 308955
total fitness: 307823
total fitness: 312889
total fitness: 311327
total fitness: 310675
total fitness: 311956
total fitness: 313912
total fitness: 310492
total fitness: 310834
total fitness: 315052
total fitness: 309721
total fitness: 308704
total fitness: 313558
total fitness: 295903
total fitness: 293525
total fitness: 291536
total fitness: 285829
total fitness: 295371
total fitness: 288669
total fitness: 286826
total fitness: 285405
total fitness: 292760
total fitness: 291200
total fitness: 293746
total fitness: 290689
total fitness: 291701
total fitness: 289079
total fitness: 292163
Generation 1: Best Fitness = 28540.500, Mutation Rate = 0.300, Crossover Rat